# Digit Recognizer

[here](https://www.kaggle.com/competitions/digit-recognizer/)

In [139]:
# imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

import keras
from keras import layers, optimizerss, metrics, callbacks

In [140]:
train_df = pd.read_csv("./data/input/train.csv")
test_df = pd.read_csv("./data/input/test.csv")

In [141]:
label = 'label'
pixels = train_df.columns.drop(label).tolist()

X = train_df[pixels].to_numpy()
y = train_df[label]

# Reshape each row to 28 * 26 (Image)
X = X.reshape(-1, 28, 28, 1)

# Normalizing/Rescaling Pixel Values
X = X / 255

# Determine number of distinct label types
n_labels = y.unique().shape[0]

# Onehot Encoding Labels
y = tf.one_hot(y, n_labels).numpy()

In [142]:
X_test_unseen = test_df[pixels].to_numpy()
X_test_unseen = X_test_unseen.reshape(-1, 28, 28, 1)
X_test_unseen = X_test_unseen / 255

In [143]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [144]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(28, 28, 1)),
        layers.Conv2D(filters=32, kernel_size=3),
        layers.MaxPooling2D(),
        layers.Conv2D(filters=16, kernel_size=3),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='sigmoid'),
        layers.Dense(10, activation='softmax')
    ]
)

#Compiling our model with appropriate loss function and optimizer
model.compile(loss = 'categorical_crossentropy', optimizer= 'adam', metrics=['categorical_accuracy'])

# Summary of the model
model.summary()

#Fitting the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 16)        4624      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 5, 5, 16)         0         
 g2D)                                                            
                                                                 
 flatten_16 (Flatten)        (None, 400)               0         
                                                                 
 dense_10 (Dense)            (None, 128)             

In [145]:
from datetime import datetime


# Predict on test unseen
y_unseen_predicted = model.predict(X_test_unseen)

# Transform labels
y_unseen_predicted = tf.argmax(y_unseen_predicted, axis = 1).numpy()

now = datetime.now().strftime("%Y%m%d_%H%M%S")

output = pd.read_csv("./data/input/sample_submission.csv", index_col=0)
output['Label'] = y_unseen_predicted

# output.to_csv(f"./data/output/digit_recognizer_{now}.csv", index=True)

875/875 [==============================] - 7s 7ms/step
